In [486]:
# Initialize Otter
import otter
grader = otter.Notebook("proj3.ipynb")

# Project 3: Data Transformation

## Due Date: Tuesday 04/13, 11:59 PM

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset---i.e. it's kind of a mess. The full dataset contains a giant `data` table of sensor readings that is many billions of readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

The schema for the database is shown below. Sometimes people think that if the data is a nice schema, then it's ready to go! We'll see about that.

<img src="files/schema.png">

## Scoring Breakdown
Question | Points
--- | ---
1a	| 1
1b  | 1
1c	| 1
1d	| 1
1e	| 1
2a	| 3
2b	| 1
3a	| 0
3b	| 1
3c	| 1
3d  | 0
3e  | 2
4a	| 2
4b	| 2
4c	| 3
5a | 1
5b | 1
5c | 3
**Total** | 25

In [487]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [488]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/template1

## Loading Up the Database
To load the database, run the following cell.

In [489]:
import subprocess
import os
import warnings

call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)

if call.stdout != "1\n":
    os.system("gunzip -c proj3.sql.gz | psql -h localhost -d template1 -f -")
else:
    warnings.warn("you need to run dropdb -h localhost ucb_buildings if you want to reload the database.")
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

<ipython-input-489-70667cec2ae0>:12: UserWarning: you need to run dropdb -h localhost ucb_buildings if you want to reload the database.
  warnings.warn("you need to run dropdb -h localhost ucb_buildings if you want to reload the database.")


Run the following cell for grading purposes.

In [490]:
!mkdir -p results

<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data

### Question 1a

Note that the `data` table, in the full database, is billions of rows. What do you notice about the design of the database schema that helps support the large amount of data?

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 1
-->

Rather than congregate all of the data under one relation and one schema, the database's schema is split up into various schemas. Another thing I noticed is the foreign key and primary key relationships that are established amongst various relations. There's also the use of normalization, where we split relations into multiple relations in a way that minimizes redundancy, enabling us to minimize updates, and delete anomolies. These are things about the design of the database schema that helps support the large amount of data.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1b

Because this data is already in a database, we have a bunch of useful high-level schematic information. But we're still missing some info we might want.

Assuming no new inserts, is there a potentially useful key in the `buildings_site_mapping` table? That is, is there a subset of columns that---at least for the provided data---is a unique ID for each row? Write a SQL query that returns a single row with one column of boolean value `true` if there is a unique ID per row, or a single row with one column of value `false` otherwise. The output column name in your query can be anything.

<!--
BEGIN QUESTION
name: q1bm
manual: true
points: 0
-->

In [491]:
%%sql result_1b <<
Select count(buildings_site_mapping) = count(grouped) AS "Has useful key?"
From buildings_site_mapping, (Select site, building, count(*)
From buildings_site_mapping
Group By site, building) AS grouped;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_1b


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q1b
points: 1
-->

In [492]:
# Do not delete/edit this cell
result_1b.DataFrame().to_csv('results/result_1b.csv', index=False)

In [493]:
grader.check("q1b")

q1b passed!

### Question 1c

The diagram claims that `buildings_site_mapping` has a many-to-many relationship with `real_estate_metadata`. Let's validate that. 

Below is an example of `json_agg` being used with a table; you will need to do this in the next two parts.

In [494]:
%%sql
SELECT b.site, json_agg(b) from buildings_site_mapping b GROUP BY b.site LIMIT 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5 rows affected.


site,json_agg
Sutardja Dai Hall,"[{'site': 'Sutardja Dai Hall', 'building': 'SUTARDJA DAI'}]"
Le Conte Hall,"[{'site': 'Le Conte Hall', 'building': 'LE CONTE'}]"
Hildebrand Hall,"[{'site': 'Hildebrand Hall', 'building': 'HILDEBRA ND'}]"
Stephens Hall,"[{'site': 'Stephens Hall', 'building': 'STEPHENS'}]"
2000 Carleton Street,"[{'site': '2000 Carleton Street', 'building': 'CARLETO2 000'}]"


Find the values of `buildings_site_mapping.building` that match multiple tuples in `real_estate_metadata.building_name`, and for each such value of `buildings_site_mapping.building` return the matches as JSON via `json_agg(real_estate_metadata)`. Your output should contain the building and the `json_agg` in that order. Order your final result by building.

In [495]:
%%sql result_1c <<
Select B.building, json_agg(R)
From buildings_site_mapping AS B, real_estate_metadata AS R
Where B.building = R.building_name
Group By B.building
Having Count(*) > 1
Order By B.building;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
9 rows affected.
Returning data to local variable result_1c


In [496]:
# Do not delete/edit this cell
result_1c.DataFrame().to_csv('results/result_1c.csv', index=False)

In [497]:
grader.check("q1c")

q1c passed!

### Question 1d

Now find examples of many matches in the opposite direction. For each `real_estate_metadata.building_name` value, find the ones that have multiple matches in `buildings_site_mapping.building`, and for each return a `json_agg` of the multiple values for `buildings_site_mapping`. Your output should contain the building name and the `json_agg` in that order. Order your final result by building name.

In [498]:
%%sql result_1d <<
Select R.building_name, json_agg(B)
From buildings_site_mapping AS B, real_estate_metadata AS R
Where B.building = R.building_name
Group By R.building_name
Having Count(*) > 1
Order By R.building_name;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
9 rows affected.
Returning data to local variable result_1d


In [499]:
# Do not delete/edit this cell
result_1d.DataFrame().to_csv('results/result_1d.csv', index=False)

In [500]:
grader.check("q1d")

q1d passed!

In [501]:
%%sql
Select R.building_name, json_agg(B)
From buildings_site_mapping AS B, real_estate_metadata AS R
Where B.building = R.building_name
Group By R.building_name
Having Count(*) > 1
Order By R.building_name;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
9 rows affected.


building_name,json_agg
ALUMNI HOUSE,"[{'site': 'Alumni House', 'building': 'ALUMNI HOUSE'}, {'site': 'Alumni House', 'building': 'ALUMNI HOUSE'}, {'site': 'Alumni House', 'building': 'ALUMNI HOUSE'}]"
CAMPBELL,"[{'site': 'Campbell Hall', 'building': 'CAMPBELL'}, {'site': 'Campbell Hall', 'building': 'CAMPBELL'}]"
EDWARDS FLD,"[{'site': 'Edwards Stadium West', 'building': 'EDWARDS FLD'}, {'site': 'Edwards Stadium East', 'building': 'EDWARDS FLD'}]"
FAC CLUB,"[{'site': 'Womens Faculty Club', 'building': 'FAC CLUB'}, {'site': ""Men's Faculty Club"", 'building': 'FAC CLUB'}]"
HAAS STU BLD,"[{'site': 'Haas School of Business', 'building': 'HAAS STU BLD'}, {'site': 'Haas School Student Services (Water)', 'building': 'HAAS STU BLD'}]"
HERTZ,"[{'site': 'Hertz and Morrison Hall', 'building': 'HERTZ'}, {'site': 'Hertz and Morrison Hall', 'building': 'HERTZ'}]"
SIMON,"[{'site': 'Boalt and Simon Hall', 'building': 'SIMON'}, {'site': 'Simpson Center', 'building': 'SIMON'}]"
SOUTH,"[{'site': 'South Hall', 'building': 'SOUTH'}, {'site': 'South Hall', 'building': 'SOUTH'}]"
SPROUL,"[{'site': 'Sproul Hall', 'building': 'SPROUL'}, {'site': 'Sproul Hall', 'building': 'SPROUL'}, {'site': 'Sproul Hall', 'building': 'SPROUL'}]"


<!-- BEGIN QUESTION -->

### Question 1e

Looking at the output of the previous question, what do you notice about the entries of the `json_agg` column? Are there any duplicates within the entries?

<!--
BEGIN QUESTION
name: q1e
manual: true
points: 1
-->

The entries of the json_agg column of result_1d show us an array of the multiple matches between tuples in the real_estate_metadata and buildings_site_mapping tables (joined on building name). In ALUMNI HOUSE, CAMPBELL, HERTZ, SOUTH, and SPROUL halls, we see duplicates within the entries of each row.

<!-- END QUESTION -->



## Question 2: Looking for Outliers in the Readings
Physical sensors like the ones generating this data are notorious for producing crazy outliers on occasion. In this section we'll do a little data cleaning of the outliers.

All the readings from all different kinds of sensors are mixed together in the `data` table. This hodgepodge of mixed readings is going to require us to do some extra work to look for outliers, which we didn't quite see in class. Let's get started.

### Question 2a: Outlier Detection

Let's find the outlying values *for each sensor id*. We'll call something an outlier if it is **3 Hampel X84 intervals** away from the median.

Specifically, create a view `labeled_data` that contains all of the columns in `data` and adds three additional columns at the far right:
  - `median` containing the median using `percentile_disc`
  - `mad` containing the mad,
  - `outlier` that contains `true` for the outlier readings and `false` for the rest. **Also,** for data points where the mad is 0, set this to `false`.

In [502]:
%%sql
CREATE OR REPLACE VIEW meedian AS
(Select id, Percentile_Disc(0.50) Within Group (Order By value) AS median
From data
Group By id);

Select *
From temp
Limit 1;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
1 rows affected.


id,median
a3d3326f-20ab-5f1d-97c7-f3084df43f06,65130.27


In [503]:
%%sql
CREATE OR REPLACE VIEW devs AS
(SELECT M.id, median, value, abs(value - median) AS dev
FROM data D, meedian M
WHERE D.id = M.id);

Select *
From devs
Limit 1;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
1 rows affected.


id,median,value,dev
a3d3326f-20ab-5f1d-97c7-f3084df43f06,65130.27,65182.74,52.470000000001164


In [504]:
%%sql
CREATE OR REPLACE VIEW maad AS
(SELECT id, Percentile_Disc(0.50) WITHIN GROUP (ORDER BY dev) AS mad
From devs
GROUP by id);

Select *
From maad;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
153 rows affected.


id,mad
a3d3326f-20ab-5f1d-97c7-f3084df43f06,22.55000000000291
a3d47b1a-985e-5395-a6ee-719dad9b580f,0.0
a3e8e405-0eed-59b6-8747-fe892a6f93de,0.8000000000000114
a43c1fa1-e6f7-56b5-ac78-badbefe43b35,0.0
a4596ae8-f98c-5c58-bdce-37b36c8af210,0.0
a46c65a7-fe0b-5151-834b-ab97d5e5678a,12.0
a46cd7f6-ede4-5a92-b9b5-3848e84e86ec,0.006899999999999906
a46d58a2-f5a2-52bd-a680-ba07f292bf70,0.0
a46fb790-028c-5d17-a3e6-7d08daec0c03,0.7299999999999613
a470ac6d-d448-522b-9b74-8ed27f24de25,0.002


In [505]:
%%sql result_2a <<
CREATE OR REPLACE VIEW labeled_data AS
(Select time, D.id, value, median, mad, 
CASE WHEN value BETWEEN (median-3*1.4826*mad) AND (median+3*1.4826*mad) THEN False
        WHEN mad = 0 THEN False
        ELSE True
    END AS outlier
From data D, meedian M, maad MD
Where D.id = M.id
And D.id = MD.id
And M.id = MD.id);

SELECT * FROM labeled_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2a


In [506]:
# Do not delete/edit this cell
result_2a.DataFrame().to_csv('results/result_2a.csv', index=False)

In [507]:
grader.check("q2a")

q2a passed!

### Question 2b: Outlier Handling (Winsorization)

In this step we'll define a view `cleaned_data` over all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). If the MAD is 0, then the cleaned value should be the same as the original value.

In [508]:
%%sql
CREATE OR REPLACE VIEW winsorized AS
(With temp AS
(Select id, max(median) AS new_med, max(mad) AS new_mad
FROM labeled_data
Group By id)
Select id, new_med, new_mad, new_med-3*1.4826*new_mad AS lower, new_med+3*1.4826*new_mad AS upper
From temp);

Select *
From new;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
153 rows affected.


id,new_med,new_mad,lower,upper
a842919a-dcd1-5d91-b1df-f70c988cabfe,5.400000000000047,0.0,5.400000000000047,5.400000000000047
a66082cb-5fc7-57fe-9deb-8645d4c2e942,0.0,0.0,0.0,0.0
a8463911-3609-5cbb-aa1b-b6eb6165568c,0.0,0.0,0.0,0.0
a8dc3e90-6bb7-56e4-bf10-bf07694da31b,7.1289999999999605,2.778999999999997,-5.231436200000026,19.489436199999947
a865712a-56b6-5e23-a61e-8be3def22661,3702057.0,1998.0,3693170.2956,3710943.7044
a6789ff0-f1da-5099-abb8-3abe65f766ab,115302.0,95.0,114879.459,115724.541
a8cf9a4c-a9ee-50e7-89f5-ca52030dbc99,122.39999999999964,0.3000000000000023,121.06565999999962,123.73433999999965
a50c665d-65f6-54eb-a83f-e56138b92b19,23.70000000000029,0.8999999999999886,19.69698000000034,27.70302000000024
a71d821d-d6e6-5ee9-86fa-2b075f13c2cc,81.0,17.0,5.3873999999999995,156.6126
a56b2aa6-6b3e-59bc-ab5a-19001053401d,496.34999999999826,0.410000000000025,494.52640199999814,498.1735979999984


In [509]:
%%sql result_2b <<
CREATE OR REPLACE VIEW ayy AS
(Select time, LD.id, value, median, mad, outlier,
 CASE WHEN outlier = False THEN value
      WHEN outlier AND value < W.lower THEN W.lower
      WHEN outlier AND value > W.upper THEN W.upper
 END AS clean_value
From labeled_data LD, winsorized W
Where LD.id = W.id);

SELECT * FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2b


In [510]:
result_2b.DataFrame().iloc[:50, -1].sum()

39270.961140000014

In [511]:
%%sql result_2b <<
CREATE OR REPLACE VIEW cleaned_data AS
(Select time, LD.id, value, median, mad, outlier,
 CASE WHEN outlier = False THEN value
      WHEN outlier AND value < W.lower THEN W.lower
      WHEN outlier AND value > W.upper THEN W.upper
 END AS clean_value
From labeled_data LD, winsorized W
Where LD.id = W.id);
SELECT * FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_2b


In [512]:
# Do not delete/edit this cell
result_2b.DataFrame().to_csv('results/result_2b.csv', index=False)

In [513]:
grader.check("q2b")

q2b passed!

<!-- BEGIN QUESTION -->

## Question 3: Entity Resolution

### Question 3a
There is a lot of mess in this dataset related to entity names. As a start, have a look at all of the distinct values in the `units` field of the `metadata` table. What do you notice about these values? Are there any duplicates?

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 0
-->

In [514]:
%%sql
Select *
From metadata
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
2 rows affected.


id,class,site,units
fb5267b9-8cf4-5710-979e-5ab7d617e6e5,Main Electric Meter (kWh),2000 Carleton Street,kWh
4e7e5872-a3f7-5300-8433-b4dfd0254cb1,Main Electric Meter Demand (kW),2000 Carleton Street,kW


These values for the most part seem to be various forms of units. Yes, there are duplicates since there are 9509 rows, but only 34 distinct unit values.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3b

Sometimes, entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns; the first column should contain the number of unique `units` values, and the second column should contain the number of unique `units` values if we ignored case.

<!--
BEGIN QUESTION
name: q3bm
manual: true
points: 0
-->

In [515]:
%%sql result_3b <<
Select count(distinct(units)) AS distinct_w_case,
count(distinct(LOWER(units))) AS distinct_wo_case
From metadata;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_3b


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [516]:
# Do not delete/edit this cell
result_3b.DataFrame().to_csv('results/result_3b.csv', index=False)

In [517]:
grader.check("q3b")

q3b passed!

<!-- BEGIN QUESTION -->

### Question 3c

Arguably we shouldn't care about these alternative unit labels, *as long as each sensor class uses a single value of `units` for all its sensor ids*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns single row with one column of value `true` if the condition in italics above holds, or a single row with one column of value `false` otherwise.

<!--
BEGIN QUESTION
name: q3cm
manual: true
points: 0
-->

In [518]:
%%sql result_3c <<
(With unit_class AS
(Select class, units, count(distinct(units)) AS total
From metadata
Group By class, units)
Select (sum(total) = count(M.class)) AS is_distinct
From unit_class, metadata M);

Select *
From distinct_check
Limit 1;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
1 rows affected.
Returning data to local variable result_3c


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [519]:
# Do not delete/edit this cell
result_3c.DataFrame().to_csv('results/result_3c.csv', index=False)

In [520]:
grader.check("q3c")

q3c passed!

In [521]:
%%sql
Select distinct(location)
From real_estate_metadata;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
15 rows affected.


location
LOS ANGELES
FRANCISC O
SYSTEMWI DE
SAN DSAIENG O
FRANCISC SOAN
SANTA CRUZ
AG FIELD STAT
IRVINE
RIVERSIDE
BERKELEY


<!-- BEGIN QUESTION -->

### Question 3d

Moving on, have a look at the `real_estate_metadata` table---starting with the distinct values in the `location` field! What do you notice about these values?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 0
-->

Many of these cities are spelled incorrectly. The letters are moved around weirdly but for the most part, I can still tell which city it which from the location names.

<!-- END QUESTION -->



### Question 3e

It turns out this table was the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition). We'll just clean up the `location` column for now, and leave you to imagine the effort to do a full cleanup of all columns.

To provide some useful utility functions, we have preloaded Postgres' extension packages for "fuzzy" string matching and trigrams for you. You can use any of the string functions in those packages if you like ([as documented here for fuzzystrmatch](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [here for pg_trgm](https://www.postgresql.org/docs/current/pgtrgm.html)).

We also created a lookup table of canonical names, `uc_locations`.

Now, using any of the string functions you like (or none at all!), create a view that has one extra column `clean_location`. That column should contain the best match from `uc_locations.loc_name`. You may find that you can't clean up everything with the string functions, so your view may have to include some specific logic for cases in the data that have to be handled "manually". You can choose to do this question in whatever manner you wish as long as your query does not use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

In [522]:
%%sql
CREATE OR REPLACE VIEW distances AS
(Select distinct(location), loc_name, levenshtein(location, loc_name) AS distance
From real_estate_metadata, uc_locations);

Select *
From distances;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
180 rows affected.


location,loc_name,distance
SAN DSAIENG O,BERKELEY,12
IRVINE,SYSTEMWIDE,8
SANTA BARBARA,MERCED,12
SANTA CRUZ,MERCED,9
BERKELEY,BERKELEY,0
MERCED,SYSTEMWIDE,8
DAVIS,MERCED,6
SAN DIEGO,SAN DIEGO,0
DAVIS,AG FIELD STAT,11
DAVIS,SAN DIEGO,7


In [523]:
%%sql
CREATE OR REPLACE VIEW lowest_dist AS
(With temp AS
(Select location, min(distance) AS lowest_dev
From distances
Group By location)

Select T.location, D.loc_name
From distances D, temp T
Where T.location = D.location
And T.lowest_dev = D.distance);

Select *
From lowest_dist;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
15 rows affected.


location,loc_name
BERKELEY,BERKELEY
SAN DIEGO,SAN DIEGO
IRVINE,IRVINE
MERCED,MERCED
FRANCISC O,SAN FRANCISCO
LOS ANGELES,LOS ANGELES
SAN DSAIENG O,SAN DIEGO
SYSTEMWI DE,SYSTEMWIDE
AG FIELD STAT,AG FIELD STAT
SANTA CRUZ,SANTA CRUZ


In [524]:
%%sql result_3e <<
Select R.location, building, building_name, address, city_name, county, category, osfg, bldg_asf,
levels, year, owner, mp_code, book_value, LD.loc_name AS clean_location
From real_estate_metadata R, lowest_dist LD
Where R.location = LD.location;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
5276 rows affected.
Returning data to local variable result_3e


In [525]:
# Do not delete/edit this cell
result_3e.DataFrame().sort_values(['clean_location', 'building', 'building_name']).iloc[::10].to_csv('results/result_3e.csv', index=False)

In [526]:
grader.check("q3e")

q3e passed!

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding missing readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so which ones? We will focus on readings that are separated by at least 30 minutes or more; readings that are \[0-30) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [Functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Have a particular look at the following:
- The [date_trunc](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. E.g. to round the `time` field to the nearest minute you can say `date_trunc('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. E.g. a 30 minute interval can be written as `interval '30 minutes'` or `'30 minutes'::interval`. See [date/time input](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on date/time types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC). That will handle all the weird periodicities of clocks and calendars for you. Pay attention to the input and output types of these functions!
- Alternatively, the [EXTRACT](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) function is sometimes handy. Note the special `EXTRACT(EPOCH FROM ...)` case. This converts a timestamp into an integer representing the number of seconds since midnight, 1/1/1970 (the dawn of [UNIX time](https://en.wikipedia.org/wiki/Unix_time)!)  You can do normal integer comparisons and arithmetic on the results.


Create a view called `gaps` that returns pairs of `data` tuples per sensor id that are separated by 30 minutes or more. The output should augment the `data` schema with three columns:
- `lagtime` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lagvalue` is the value of the previous reading for that sensor
- `timediff` is the difference in quantized time between this reading and the previous reading

In [527]:
%%sql
CREATE OR REPLACE VIEW updated_time AS
(Select *, date_trunc('minute', time) AS quantized
From data);

Select *
From updated_time
Limit 10;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
10 rows affected.


time,id,value,quantized
2018-06-12 23:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.74,2018-06-12 23:00:00+00:00
2018-06-12 22:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.51,2018-06-12 22:45:00+00:00
2018-06-12 22:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.27,2018-06-12 22:30:00+00:00
2018-06-12 22:15:10+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65182.02,2018-06-12 22:15:00+00:00
2018-06-12 22:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.77,2018-06-12 22:00:00+00:00
2018-06-12 21:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.54,2018-06-12 21:45:00+00:00
2018-06-12 21:30:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.3,2018-06-12 21:30:00+00:00
2018-06-12 21:15:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65181.04,2018-06-12 21:15:00+00:00
2018-06-12 21:00:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.8,2018-06-12 21:00:00+00:00
2018-06-12 20:45:09+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65180.57,2018-06-12 20:45:00+00:00


In [528]:
%%sql
CREATE OR REPLACE VIEW gaps_temp AS
(With times AS
(Select *,
LAG(quantized, 1) OVER (PARTITION BY id ORDER BY quantized) AS lagtime,
LAG(value, 1) OVER (PARTITION BY id ORDER BY quantized) AS lagvalue
From updated_time)

Select quantized AS time, id, value, lagtime, lagvalue,
(EXTRACT(EPOCH FROM quantized - lagtime))/60 AS timediff
From times);

Select *
From gaps_temp
Limit 10;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
10 rows affected.


time,id,value,lagtime,lagvalue,timediff
2018-06-07 00:00:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65085.99,None,None,None
2018-06-07 00:15:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.16,2018-06-07 00:00:00+00:00,65085.99,15.0
2018-06-07 00:30:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.35,2018-06-07 00:15:00+00:00,65086.16,15.0
2018-06-07 00:45:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.52,2018-06-07 00:30:00+00:00,65086.35,15.0
2018-06-07 01:00:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.71,2018-06-07 00:45:00+00:00,65086.52,15.0
2018-06-07 01:15:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65086.89,2018-06-07 01:00:00+00:00,65086.71,15.0
2018-06-07 01:30:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.07,2018-06-07 01:15:00+00:00,65086.89,15.0
2018-06-07 01:45:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.25,2018-06-07 01:30:00+00:00,65087.07,15.0
2018-06-07 02:00:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.43,2018-06-07 01:45:00+00:00,65087.25,15.0
2018-06-07 02:15:00+00:00,a3d3326f-20ab-5f1d-97c7-f3084df43f06,65087.62,2018-06-07 02:00:00+00:00,65087.43,15.0


In [529]:
%%sql result_4a <<
CREATE OR REPLACE VIEW gaps AS
(Select *
From gaps_temp
Where timediff >= 30)
;
SELECT * FROM gaps ORDER BY id, time LIMIT 10;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
10 rows affected.
Returning data to local variable result_4a


In [530]:
# Do not delete/edit this cell
result_4a.DataFrame().to_csv('results/result_4a.csv', index=False)

In [531]:
grader.check("q4a")

q4a passed!

### Question 4b: Creating tuples for the missing readings
Now we need to manufacture new tuples to fill in the gaps. For example, if you had a tuple from id `abc` timestamped at 1PM today and the next tuple in time from `abc` was timestamped at 1:45PM, you'll need to manufacture two new tuples with id `abc` and `NULL` values: one timestamped at 1:15PM and another timestamped at 1:30PM. We will worry about replacing the `NULL` values in the next step.

To manufacture tuples not related to stored data in the database, we'll need to use a *table-valued function* as we did in lecture 12 (when we manufactured data from a normal distribution). The table-valued function we want here is `generate_series` [(documented here)](https://www.postgresql.org/docs/current/functions-srf.html), which we will use to generate *and sequentially timestamp* the right number of tuples to match the number of tuples we found missing.

To get a feel for `generate_series`, consider the following simple query that generates a table of integers with intervals of size 3 between them.

In [532]:
%%sql
SELECT *
  FROM generate_series(1, 10, 3);

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
4 rows affected.


generate_series
1
4
7
10


Now, we can use `generate_series` in a `LATERAL JOIN` query: for each tuple on the left of the `LATERAL` it will produce a series based on the values of that tuple. So for example, we can generate 2 tuples for each tuple of `uc_locations` as follows:

In [533]:
%%sql
SELECT loc_id, loc_name, length(loc_name), newval
  FROM uc_locations u, 
       LATERAL generate_series(length(loc_name), length(loc_name) + 2, 2) AS newval;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
24 rows affected.


loc_id,loc_name,length,newval
1,AG FIELD STAT,13,13
1,AG FIELD STAT,13,15
2,BERKELEY,8,8
2,BERKELEY,8,10
3,DAVIS,5,5
3,DAVIS,5,7
4,IRVINE,6,6
4,IRVINE,6,8
5,LOS ANGELES,11,11
5,LOS ANGELES,11,13


Notice how the 2 values it generates are the length of the `loc_name`, and the length + 2. You might want to play with the query above to make sure you understand the documentation for `generate_series` and `LATERAL`.

Ok, on to your task!

Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, with `NULL` as the value. You probably want to use your `gaps` view as well as `generate_series` to do this!

In [534]:
%%sql
DROP VIEW IF EXISTS almost_complete;
CREATE OR REPLACE VIEW almost_complete AS
(With temp AS
(Select *
From gaps, LATERAL generate_series(lagtime, time, '15 minutes') AS new_time
Where EXTRACT(EPOCH FROM time - new_time)/60 >= 15
And lagtime != new_time)
 
Select id, new_time AS time, 
CASE WHEN lagtime != new_time THEN NULL
ELSE value
END AS value
From temp
Union
Select id, time, value From gaps_temp
Order By id, time);

Select count(*)
From almost_complete
Limit 20;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
Done.
1 rows affected.


count
102386


In [535]:
%%sql
Select *
From complete
Where value is null
Limit 10;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
10 rows affected.


id,time,value
a3d47b1a-985e-5395-a6ee-719dad9b580f,2018-06-13 09:00:00+00:00,None
a3e8e405-0eed-59b6-8747-fe892a6f93de,2018-06-09 04:00:00+00:00,None
a46fb790-028c-5d17-a3e6-7d08daec0c03,2018-06-13 09:00:00+00:00,None
a470ac6d-d448-522b-9b74-8ed27f24de25,2018-06-09 04:00:00+00:00,None
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 00:30:00+00:00,None
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 01:15:00+00:00,None
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 17:30:00+00:00,None
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 18:30:00+00:00,None
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:00:00+00:00,None
a48e47ab-bbcd-5b39-aaaa-47fbaafe166e,2018-06-07 19:30:00+00:00,None


In [536]:
%%sql result_4b <<
CREATE OR REPLACE VIEW complete AS
(With temp AS
(Select *
From gaps, LATERAL generate_series(lagtime, time, '15 minutes') AS new_time
Where EXTRACT(EPOCH FROM time - new_time)/60 >= 15
And lagtime != new_time)
 
Select id, new_time AS time, 
CASE WHEN lagtime != new_time THEN NULL
ELSE value
END AS value
From temp
Union
Select id, time, value From gaps_temp
Order By id, time);

SELECT * FROM complete ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
100 rows affected.
Returning data to local variable result_4b


In [537]:
# Do not delete/edit this cell
result_4b.DataFrame().to_csv('results/result_4b.csv', index=False)

In [538]:
grader.check("q4b")

q4b passed!

### Question 4c: Linear Interpolation
*Note: If you struggled with Steps 1 and 2 of this problem, you can use our table `complete_provided` instead of your `complete` table in Step 3.*

Now, given the `complete` view or the `complete_provided` table, the remaining task is to do linear interpolation to fill in the missing values we manufactured in Step 2. We have code from Lecture 13 we can reuse here! In particular, your database already includes the UDA `coalesce_agg` we used in lecture.

But note that in Lecture 13's example of linear interpolation we had a field called `feature_id` that had two convenient properties:
1. `feature_id` was used to order *all* the records in the table. By contrast, here the ordering we care about is the series of timestamps for each sensor `id` *independently*.
2. `feature_id` was a gap-free sequence of incrementing integers. Coupled with the previous point, that allowed us to use arithmetic on `feature_id` to calculate the distance between records in order in the "backward" pass. We don't have any such field handy here, so you'll have to find some other way (hint: window function!) to achieve the same effect where you need it.

These two points will require you to adapt the linear interpolation code from class to work here.

Create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is non-NULL, otherwise an interpolated value based on linear interpolation *per sensor id over time*. The three cells below correspond to the forward, backward, and final passes from lecture.

In [539]:
%%sql
CREATE OR REPLACE VIEW w_rank AS
(Select *, RANK() OVER (PARTITION BY ID ORDER BY TIME) AS ranking
From complete);

Select *
From w_rank
Limit 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5 rows affected.


id,time,value,ranking
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:00+00:00,65085.99,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:00+00:00,65086.16,2
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:00+00:00,65086.35,3
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:00+00:00,65086.52,4
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:00+00:00,65086.71,5


In [540]:
%%sql
CREATE OR REPLACE VIEW forward AS
(Select *,
    SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END) OVER (PARTITION BY id ORDER BY ranking) AS run,
    coalesce_agg(value) OVER (PARTITION BY id ORDER BY ranking) AS run_start,
    CASE WHEN value IS NULL
        THEN lead(value, 1) OVER (PARTITION BY id ORDER BY ranking)
        ELSE NULL
    END AS nextval
From w_rank);

Select *
From forward
Limit 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5 rows affected.


id,time,value,ranking,run,run_start,nextval
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:00+00:00,65085.99,1,1,65085.99,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:00+00:00,65086.16,2,2,65086.16,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:00+00:00,65086.35,3,3,65086.35,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:00+00:00,65086.52,4,4,65086.52,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:00+00:00,65086.71,5,5,65086.71,None


In [541]:
%%sql
CREATE OR REPLACE VIEW backward AS
(Select *,
     CASE WHEN value IS NOT NULL THEN value
         ELSE coalesce_agg(nextval) OVER (PARTITION BY id, run ORDER BY ranking DESC)
     END AS run_end,
     count(*) OVER (PARTITION BY id, run) AS run_size,
     1 + ranking - (min(ranking) OVER (PARTITION BY id, run)) AS run_rank 
 From forward);

Select *
From backward
Limit 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5 rows affected.


id,time,value,ranking,run,run_start,nextval,run_end,run_size,run_rank
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:00+00:00,65085.99,1,1,65085.99,None,65085.99,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:00+00:00,65086.16,2,2,65086.16,None,65086.16,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:00+00:00,65086.35,3,3,65086.35,None,65086.35,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:00+00:00,65086.52,4,4,65086.52,None,65086.52,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:00+00:00,65086.71,5,5,65086.71,None,65086.71,1,1


In [542]:
%%sql
DROP VIEW IF EXISTS likely_data;
CREATE OR REPLACE VIEW likely_data AS
(Select *,
     run_start + (run_rank-1)*((run_end - run_start)/(run_size)) AS interpolated
 From backward);

Select *
From likely_data
Limit 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
Done.
5 rows affected.


id,time,value,ranking,run,run_start,nextval,run_end,run_size,run_rank,interpolated
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:00+00:00,65085.99,1,1,65085.99,None,65085.99,1,1,65085.99
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:00+00:00,65086.16,2,2,65086.16,None,65086.16,1,1,65086.16
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:00+00:00,65086.35,3,3,65086.35,None,65086.35,1,1,65086.35
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:00+00:00,65086.52,4,4,65086.52,None,65086.52,1,1,65086.52
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:00+00:00,65086.71,5,5,65086.71,None,65086.71,1,1,65086.71


In [543]:
%%sql
CREATE OR REPLACE VIEW forward AS
(Select *,
    SUM(CASE WHEN value IS NULL THEN 0 ELSE 1 END) OVER (PARTITION BY id ORDER BY ranking) AS run,
    coalesce_agg(value) OVER (PARTITION BY id ORDER BY ranking) AS run_start,
    CASE WHEN value IS NULL
        THEN lead(value, 1) OVER (PARTITION BY id ORDER BY ranking)
        ELSE NULL
    END AS nextval
From w_rank);

Select *
From forward
Limit 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5 rows affected.


id,time,value,ranking,run,run_start,nextval
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:00+00:00,65085.99,1,1,65085.99,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:00+00:00,65086.16,2,2,65086.16,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:00+00:00,65086.35,3,3,65086.35,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:00+00:00,65086.52,4,4,65086.52,None
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:00+00:00,65086.71,5,5,65086.71,None


In [544]:
%%sql
CREATE OR REPLACE VIEW backward AS
(Select *,
     CASE WHEN value IS NOT NULL THEN value
         ELSE coalesce_agg(nextval) OVER (PARTITION BY id, run ORDER BY ranking DESC)
     END AS run_end,
     count(*) OVER (PARTITION BY id, run) AS run_size,
     1 + ranking - (min(ranking) OVER (PARTITION BY id, run)) AS run_rank 
 From forward);

Select *
From backward
Limit 5;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
5 rows affected.


id,time,value,ranking,run,run_start,nextval,run_end,run_size,run_rank
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:00:00+00:00,65085.99,1,1,65085.99,None,65085.99,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:15:00+00:00,65086.16,2,2,65086.16,None,65086.16,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:30:00+00:00,65086.35,3,3,65086.35,None,65086.35,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 00:45:00+00:00,65086.52,4,4,65086.52,None,65086.52,1,1
a3d3326f-20ab-5f1d-97c7-f3084df43f06,2018-06-07 01:00:00+00:00,65086.71,5,5,65086.71,None,65086.71,1,1


In [545]:
%%sql result_4c <<
DROP VIEW IF EXISTS likely_data;
CREATE OR REPLACE VIEW likely_data AS
(Select id, time, value, run, run_start, nextval, run_end, run_size, run_rank,
     run_start + (run_rank-1)*((run_end - run_start)/(run_size)) AS interpolated
 From backward);

Select *
From likely_data
Limit 5;

SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
Done.
5 rows affected.
100 rows affected.
Returning data to local variable result_4c


In [546]:
# Do not delete/edit this cell
result_4c.DataFrame().to_csv('results/result_4c.csv', index=False)

In [547]:
grader.check("q4c")

q4c passed!

## Question 5: Granularity Transforms
In this question we will write a roll-up query on an ontology. This requires a bit of background explanation.

### The Brick Ontology
The ongoing [Software-Defined Buildings](http://sdb.cs.berkeley.edu/sdb/) research project at Berkeley has led the development of a standard ontology for building metadata called [Brick](https://docs.brickschema.org/intro.html) that is getting a fair bit of attention in the world of IoT. Like many ontologies, it is represented as triples `(subject, predicate, object)`. In our database the Brick ontology has been stored in a table called `ontology`.

### The `SubClassOf` Predicate and the `Sensor` Class
We are interested in readings from different classes of sensor devices. More specifically, we are interested in rows from the `metadata` table whose `metadata.class` entry maps to an `ontology` subject $s$, and that subject is in an ontology tuple `(`$s$`, http://www.w3.org/2000/01/rdf-schema#subClassOf, https://brickschema.org/schema/Brick#Sensor)`. Then we know that the sensor in `metadata` belongs to a sub-class of `Sensor`. 

The diagram below shows a few of the `subject`s and `object`s from `ontology` in ovals. There is a dark arrow between two ovals if there is a corresponding row in `ontology`. The sub-classes of `Sensor` in the diagram are shown in yellow; we'll call them "Sensor children".

### The `transitive_subClassOf` Relation
Intuitively, the subclasses of the "Sensor children" are also themselves sensors; and the subclasses of those classes are also sensors, and so on. That is, we're really interested in the [transitive closure](https://en.wikipedia.org/wiki/Transitive_closure) of the `subClassOf` predicate. We can form transitive "chains" like this by joining `ontology` with itself. For example, the query 
```sql
SELECT o1.subject, o2.object 
  FROM ontology o1 INNER JOIN ontology o2 ON o1.object = o2.subject
 WHERE o1.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'
   AND o2.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'
``` 
forms edges between the endpoints of chains of length 2.

Extending this example, computing edges of length 3 requires joining 3 references to `ontology`, and so on. To form all chains of *arbitrary* length requires the use of a *recursive query*---something we haven't learned in this class. So we have provided you a materialized view called `transitive_subClassOf` that provides the result of that recursive query. It contains tuples of the form `(object, subject, hops, path)` where `subject` and `object` are connected transitively in `ontology` via one or more `subClassOf` predicates described above. `path` is a Postgres array type that shows the transitive path of class names through the ontology from `subject` to `object`, and `hops` is the length of that path. In the figure below, there is an aquamarine edge from one node to another if there is a row in `transitive_subClassOf` for that pair. For example, there is a row:

| object | subject | hops | path |
| :-- | :-- | :-- | :-- |
| `https://brickschema.org/schema/Brick#Sensor` | `https://brickschema.org/schema/Brick#CO2_Level_Sensor` | `3`  | `{https://brickschema.org/schema/Brick#CO2_Level_Sensor,`
| | | | `https://brickschema.org/schema/Brick#CO2_Sensor,`
| | | | `https://brickschema.org/schema/Brick#Particulate_Matter_Sensor,`
| | | | `https://brickschema.org/schema/Brick#Sensor}`

<img src="files/subClass.png">

*Just for fun: If you're curious about the recursive query that computes this view, you can issue the command `\d+ transitive_subClassOf` to Postgres. You may also want to read the documentation for SQL's [WITH RECURSIVE](https://www.postgresql.org/docs/9.1/queries-with.html) clause as implemented in Postgres.*

<!-- BEGIN QUESTION -->

### Question 5a

We want to check the graph properties of the `subClassOf` predicate. It would be confusing if the `subClassOf` predicate had cycles!

Write a query on `transitive_subClassOf` to check for cycles. Ask yourself this: what property in `transitive_subClassOf` would be a "witness" to a cycle?? Your query should return one row of one boolean column: `true` if the predicate has cycles, `false` otherwise.

<!--
BEGIN QUESTION
name: q5am
manual: true
points: 0
-->

In [548]:
%%sql result_5a <<
With temp AS
(Select "object" = "subject" AS is_cycle 
From transitive_subClassOf)

Select count(*) > 0 AS has_cycles
From temp
Where is_cycle = True;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_5a


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q5a
points: 1
-->

In [549]:
# Do not delete/edit this cell
result_5a.DataFrame().to_csv('results/result_5a.csv', index=False)

In [550]:
grader.check("q5a")

q5a passed!

<!-- BEGIN QUESTION -->

### Question 5b

Assuming it's not cyclic, the next question is whether the `subClassOf` predicate forms *tree-shaped* connections only. The signature of a tree is that each node has at most one outbound edge (pointing to its "parent"). If any node has multiple outbound edges, the predicate forms a more general directed acyclic graph (a DAG). So we are looking to see if each subject is in a `subClassOf` predicate *with at most one object* (the single parent in the tree). 

Write a query that returns `true` if each subject is in a `subClassOf` predicate with at most one object, and `false` otherwise.

<!--
BEGIN QUESTION
name: q5bm
manual: true
points: 0
-->

In [551]:
%%sql
With temp AS
(Select * From transitive_subClassOf t1, transitive_subClassOf t2
 Where t1.object = t2.object And t1.subject = t2.subject And t1.hops != t2.hops)

Select count(*) = 0 AS is_dag
From temp;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.


is_dag
False


In [552]:
%%sql result_5b <<
Select count(distinct("object", "subject")) = count(transitive_subClassOf) AS is_dag
From transitive_subClassOf;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
1 rows affected.
Returning data to local variable result_5b


<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q5b
points: 1
-->

In [553]:
# Do not delete/edit this cell
result_5b.DataFrame().to_csv('results/result_5b.csv', index=False)

In [554]:
grader.check("q5b")

q5b passed!

### Question 5c

Now that we understand the graph properties of the ontology, let's use the ontology to do a roll-up as intended. We're interested in the number of unique sensor `id`s from `metadata` that are transitively in subclasses of each "Sensor child" class. To compute this, you will have to associate each `metadata.id` with a matching `brickclass` *(if there is one!)*, and use the `transitive_subClassOf` view to identify all "Sensor children" for which the matching `brickclass` is transitively in a `subClass`. (If the `subClass` predicate is a DAG, then each `id` should be counted for *all* the direct subclasses of `Sensor` that it's transitively underneath.)

Write a query that returns tuples of the form `(sensor_child, count)` that returns for each "Sensor child" the count of *distinct* `metadata.id` entries that are subclasses of that "Sensor child" class. Only output tuples that have a matching `brickclass`.

In [555]:
%%sql
Select *
From metadata
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
2 rows affected.


id,class,site,units
fb5267b9-8cf4-5710-979e-5ab7d617e6e5,Main Electric Meter (kWh),2000 Carleton Street,kWh
4e7e5872-a3f7-5300-8433-b4dfd0254cb1,Main Electric Meter Demand (kW),2000 Carleton Street,kW


In [556]:
%%sql
Select *
From ontology
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
2 rows affected.


subject,predicate,object
f4a92336c20b04abaa6478cdd5156f7cab2672,http://www.w3.org/1999/02/22-rdf-syntax-ns#rest,f4a92336c20b04abaa6478cdd5156f7cab2673
f4a92336c20b04abaa6478cdd5156f7cab2217,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,f4a92336c20b04abaa6478cdd5156f7cab93


In [557]:
%%sql
Select *
From mapping
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
2 rows affected.


rawname,brickclass
225 KVA Xfmr - Panel LNDB-2 Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor
Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor


In [558]:
%%sql
CREATE OR REPLACE VIEW brick_match AS
(Select id, class, brickclass
 From metadata MD, mapping MP
 Where MD.class = MP.rawname);

CREATE OR REPLACE VIEW filtered AS
(Select *
 From ontology
 Where object = 'https://brickschema.org/schema/Brick#Sensor'
 And predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf');

Select *
From brick_match
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
Done.
2 rows affected.


id,class,brickclass
fb5267b9-8cf4-5710-979e-5ab7d617e6e5,Main Electric Meter (kWh),https://brickschema.org/schema/Brick#Energy_Sensor
4e7e5872-a3f7-5300-8433-b4dfd0254cb1,Main Electric Meter Demand (kW),https://brickschema.org/schema/Brick#Active_Power_Sensor


In [559]:
%%sql
Select distinct(subject) From filtered;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
38 rows affected.


subject
https://brickschema.org/schema/Brick#Adjust_Sensor
https://brickschema.org/schema/Brick#Air_Grains_Sensor
https://brickschema.org/schema/Brick#Angle_Sensor
https://brickschema.org/schema/Brick#Capacity_Sensor
https://brickschema.org/schema/Brick#Conductivity_Sensor
https://brickschema.org/schema/Brick#Contact_Sensor
https://brickschema.org/schema/Brick#Current_Sensor
https://brickschema.org/schema/Brick#Demand_Sensor
https://brickschema.org/schema/Brick#Dewpoint_Sensor
https://brickschema.org/schema/Brick#Direction_Sensor


In [560]:
%%sql
CREATE OR REPLACE VIEW grandchildren AS
(Select * From transitive_subClassOf
 Where object IN (Select distinct(subject) From filtered));

Select *
From grandchildren
Order By hops Desc
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
2 rows affected.


object,subject,hops,path
https://brickschema.org/schema/Brick#Temperature_Sensor,https://brickschema.org/schema/Brick#High_Temperature_Hot_Water_Return_Temperature_Sensor,4,"['https://brickschema.org/schema/Brick#High_Temperature_Hot_Water_Return_Temperature_Sensor', 'https://brickschema.org/schema/Brick#Hot_Water_Return_Temperature_Sensor', 'https://brickschema.org/schema/Brick#Return_Water_Temperature_Sensor', 'https://brickschema.org/schema/Brick#Water_Temperature_Sensor', 'https://brickschema.org/schema/Brick#Temperature_Sensor']"
https://brickschema.org/schema/Brick#Temperature_Sensor,https://brickschema.org/schema/Brick#Low_Outside_Air_Temperature_Enable_Differential_Sensor,4,"['https://brickschema.org/schema/Brick#Low_Outside_Air_Temperature_Enable_Differential_Sensor', 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Enable_Differential_Sensor', 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor', 'https://brickschema.org/schema/Brick#Air_Temperature_Sensor', 'https://brickschema.org/schema/Brick#Temperature_Sensor']"


In [561]:
%%sql
CREATE OR REPLACE VIEW relations AS
(Select id, object, subject
 From brick_match B, grandchildren G
 Where B.brickclass = G.subject);

Select *
From relations
Limit 2;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
Done.
2 rows affected.


id,object,subject
9439c96d-bb61-5353-8cec-e0c8f0868378,https://brickschema.org/schema/Brick#Power_Sensor,https://brickschema.org/schema/Brick#Active_Power_Sensor
f48add70-ea40-5e9d-812c-153ba886d5ff,https://brickschema.org/schema/Brick#Power_Sensor,https://brickschema.org/schema/Brick#Active_Power_Sensor


In [562]:
%%sql
Select object AS sensor_child, count(*)
From relations
Group By object;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
2 rows affected.


sensor_child,count
https://brickschema.org/schema/Brick#Demand_Sensor,92
https://brickschema.org/schema/Brick#Power_Sensor,3106


In [563]:
%%sql result_5c <<
Select object AS sensor_child, count(*)
From relations
Group By object;

   postgresql://jovyan@127.0.0.1:5432/template1
 * postgresql://jovyan@127.0.0.1:5432/ucb_buildings
2 rows affected.
Returning data to local variable result_5c


In [564]:
# Do not delete/edit this cell
result_5c.DataFrame().to_csv('results/result_5c.csv', index=False)

In [565]:
grader.check("q5c")

q5c passed!

## Congratulations! You have finished Project 3.

Run the following cell to zip the results of your queries. You will also need to run the export cell at the end of the notebook. **For submission on Gradescope, you will need to submit both the proj3.zip file generated by the export cell and the results.zip file generated by the following cell.**

In [566]:
!zip -r results.zip results

updating: results/ (stored 0%)
updating: results/result_1b.csv (stored 0%)
updating: results/result_1c.csv (deflated 84%)
updating: results/result_1d.csv (deflated 71%)
updating: results/result_2a.csv (deflated 92%)
updating: results/result_3b.csv (deflated 23%)
updating: results/result_2b.csv (deflated 92%)
updating: results/result_3c.csv (stored 0%)
updating: results/result_3e.csv (deflated 74%)
updating: results/result_4a.csv (deflated 72%)
updating: results/result_4b.csv (deflated 90%)
updating: results/result_4c.csv (deflated 87%)
updating: results/result_5b.csv (stored 0%)
updating: results/result_5a.csv (stored 0%)
updating: results/result_5c.csv (deflated 40%)


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [567]:
grader.check_all()

q1b passed!

q1c passed!

q1d passed!

q2a passed!

q2b passed!

q3b passed!

q3c passed!

q3e passed!

q4a passed!

q4b passed!

q4c passed!

q5a passed!

q5b passed!

q5c passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()